In [2]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.metrics import make_scorer, recall_score, accuracy_score, f1_score, precision_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from scipy.stats import ttest_rel
from sklearn.model_selection import KFold

sys.path.append(os.path.abspath('..'))
from processamento import df

## 4.3.1 Novo atributo: RespDisease

In [3]:
# Definir as doenças respiratórias
respiratory_conditions = ['Asthma', 'Chronic obstructive pulmonary disease']

# Criar o novo atributo RespDisease: 1 se for doença respiratória, 0 caso contrário
df['RespDisease'] = df['Outcome'].apply(lambda x: 1 if x in respiratory_conditions else 0)

# Obter dataframe apenas com valores númericos
df_num = df.select_dtypes(include=[np.number])

# Contagem específica
asthma_count = df['Outcome'].value_counts().get('Asthma', 0)
copd_count = df['Outcome'].value_counts().get('Chronic obstructive pulmonary disease', 0)

print(f"Asthma: {asthma_count}")
print(f"Chronic obstructive pulmonary disease: {copd_count}")

# Confirmar com RespDisease
print(f"Doenças respiratórias: {(df['RespDisease'] == 1).sum()}")


Asthma: 10254
Chronic obstructive pulmonary disease: 8485
Doenças respiratórias: 18739


# **4.3.2 Modelos de previsão de doenças respiratórias**

### **Nota**: A 4.3.2 está realizada juntamente com a 4.3.3

In [ ]:
# Define a função para calcular a especificidade
# A especificidade mede a proporção de negativos corretamente identificados (i.e., verdadeiros negativos)
# Como o recall_score assume por padrão o rótulo positivo como 1, aqui define pos_label=0 
# para que o cálculo seja feito sobre a classe negativa (ausência de doença)
def specificity_score(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=0)

# Cria um scorer customizado compatível com funções de validação, como cross_validate e GridSearchCV
specificity_scorer = make_scorer(specificity_score)

# Define as métricas a serem calculadas durante a validação dos modelos
# - Accuracy: Proporção total de classificações corretas
# - Sensitivity (Recall): Capacidade de identificar corretamente os positivos (doentes)
# - Specificity: Capacidade de identificar corretamente os negativos (não-doentes)
# - F1: Média harmónica entre precisão e recall (útil para classes desbalanceadas)
scoring = {
    'Accuracy': 'accuracy',
    'Sensitivity': 'recall',
    'Specificity': specificity_scorer,
    'F1': 'f1'
}

# Define a matriz de atributos (X) e o vetor alvo (y)
# - Remove a variável alvo RespDisease de X, para não ser usada como input
# - y é a variável binária indicando presença ou ausência de doenças respiratórias
X = df_num.drop(columns=['RespDisease'])
y = df_num['RespDisease']

> * Para todos os modelos, foi utilizada uma validação cruzada com otimização dos hiperparâmetros través do método GridSearchCV dividida em 5 partições (5-fold cross-validation). 
> 
> * Escolheu-se a validação cruzada com 5 partições, por esta ser bastante utilizada, permite estimar o desempenho dos modelos de forma fiável e mantem a eficiência computacional.


#### a. Modelo árvore de decisão

#### Testes Manuais

In [ ]:
# Divide os dados em 5 conjuntos (folds), com os dados baralhados antes.
# O parâmetro random_state assegura que a divisão é reprodutível (mesma em cada execução).
# Usado para criar os folds de validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Função para executar a validação cruzada com os hiperparâmetros (max_depth e min_samples_split) definidos
def run_cv(max_depth, min_samples_split):
    # Listas para armazenar os resultados das métricas em cada fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Percorre os 5 folds da validação cruzada
    for train_idx, val_idx in kf.split(X):
        # Divide os dados em conjuntos de treino e validação
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Cria e treina o modelo de árvore de decisão com os hiperparâmetros definidos
        model = DecisionTreeClassifier(
            max_depth=max_depth,                     # Profundidade máxima da árvore
            min_samples_split=min_samples_split,     # Número mínimo de amostras necessárias para dividir um nó
            random_state=42,                         # Garantir consistência nos resultados
        )
        
        # Treina o modelo com os dados de treino
        model.fit(X_train, y_train)

        # Faz previsões com os dados de validação
        y_pred = model.predict(X_val)

        # Calcula as métricas de desempenho e guarda os resultados
        accuracy_scores.append(accuracy_score(y_val, y_pred))
        precision_scores.append(precision_score(y_val, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_val, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_val, y_pred, zero_division=0))

    # Mostra os resultados da média e desvio padrão das métricas
    print(f"\nmax_depth={max_depth}, min_samples_split={min_samples_split}")
    print(f"Accuracy: Mean = {np.mean(accuracy_scores):.3f} Std = {np.std(accuracy_scores):.3f}")
    print(f"Precision: Mean = {np.mean(precision_scores):.3f} Std = {np.std(precision_scores):.3f}")
    print(f"Recall: Mean = {np.mean(recall_scores):.3f} Std = {np.std(recall_scores):.3f}")
    print(f"F1 Score: Mean = {np.mean(f1_scores):.3f} Std = {np.std(f1_scores):.3f}")

# Teste manual 1
print("Teste manual 1:")
run_cv(max_depth=None, min_samples_split=2)

# Teste manual 2
print()
print("Teste manual 2:")
run_cv(max_depth=5, min_samples_split=2)

# Teste manual 3
print()
print("Teste manual 3:")
run_cv(max_depth=10, min_samples_split=5)

Teste manual 1:

max_depth=None, min_samples_split=2
Accuracy: Mean = 0.798 Std = 0.004
Precision: Mean = 0.743 Std = 0.011
Recall: Mean = 0.719 Std = 0.006
F1 Score: Mean = 0.731 Std = 0.005

Teste manual 2:

max_depth=5, min_samples_split=2
Accuracy: Mean = 0.776 Std = 0.005
Precision: Mean = 0.869 Std = 0.004
Recall: Mean = 0.485 Std = 0.008
F1 Score: Mean = 0.623 Std = 0.007

Teste manual 3:

max_depth=10, min_samples_split=5
Accuracy: Mean = 0.797 Std = 0.006
Precision: Mean = 0.915 Std = 0.021
Recall: Mean = 0.516 Std = 0.007
F1 Score: Mean = 0.659 Std = 0.006


> * O teste de max_depth=None e min_samples_split=2 apresentou o melhor equilíbrio entre precisão e sensibilidade, resultando no maior F1 Score.
>  
> * Apesar de modelos mais simples serem mais precisos, sacrificam a capacidade de identificar corretamente os casos positivos. Assim, a árvore mais complexa revelou-se mais eficaz para este problema.

In [ ]:
# Hiperparâmetros do árvore de decisão
dt_model_params = {
    'max_depth': [None, 5, 10, 20], 
    'min_samples_split': [2, 5, 10] 
}

# Otimizar hiperparâmetros do DecisionTreeClassifier
grid = GridSearchCV(
    DecisionTreeClassifier(),
    dt_model_params,
    cv=5,                                # 5-fold cross-validation
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

# Ajustar (treinar) o modelo com os dados
grid.fit(X, y)

# Obter os melhores parâmetros e a melhor pontuação
dt_best_params = grid.best_params_
dt_best_score = -grid.best_score_.round(2)
print(f"Best Decision Tree Parameters: {dt_best_params}")
print(f"Best Score: {dt_best_score}")

# Treinar o modelo com os melhores parâmetros
dt_model = DecisionTreeClassifier(**dt_best_params)

# Avaliar desempenho do modelo 
scores = cross_validate(dt_model, X, y, cv=5, scoring=scoring, n_jobs=-1)

# Calcular as métricas de avaliação
dt_metrics = {}
for metric in scoring.keys():
    dt_metrics[metric] = (
        np.mean(scores[f'test_{metric}']), 
        np.std(scores[f'test_{metric}'])
    )

# Exibir as métricas de avaliação do modelo
print("Decision Tree Evaluation:")
for metric, (mean, std) in dt_metrics.items():
    print(f"{metric}: Mean = {mean:.3f}, Std = {std:.3f}")

# Guarda o modelo 
decision_tree = dt_model

Best Decision Tree Parameters: {'max_depth': 10, 'min_samples_split': 5}
Best Score: 0.22
Decision Tree Evaluation:
Accuracy: Mean = 0.782, Std = 0.024
Sensitivity: Mean = 0.510, Std = 0.049
Specificity: Mean = 0.949, Std = 0.035
F1: Mean = 0.640, Std = 0.042


> * Melhores hiperparâmetros identificados foram max_depth=10 e min_samples_split=5;
> 
> * Obteve-se um erro médio absoluto (MAE) de 0.22, o que indica que, em média, o desvio entre as previsões do modelo e os valores reais da variável alvo foi de 22%.
> 
> * Este valor mostra um desempenho aceitável considerando a complexidade do fenómeno estudado e as possíveis limitações dos dados.

#### b. Modelo rede neuronal

In [9]:
# Divide os dados em 5 conjuntos (folds), com os dados baralhados antes.
# O parâmetro random_state assegura que a divisão é reprodutível (mesma em cada execução).
# Usado para criar os folds de validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Função para executar a validação cruzada com os hiperparâmetros (n_neighbors e weights) definidos
def run_nn_cv(hidden_layer_sizes, learning_rate_init):
    # Listas para armazenar os resultados das métricas em cada fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Percorre os 5 folds da validação cruzada
    for train_idx, val_idx in kf.split(X):
        # Divide os dados em conjuntos de treino e validação
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Cria e treina o modelo de rede neural com os hiperparâmetros definidos
        model = make_pipeline(
            # Normaliza os dados antes de treinar a rede neural
            StandardScaler(),
            # Cria a rede neural com os hiperparâmetros especificados
            MLPClassifier(
                hidden_layer_sizes=hidden_layer_sizes,  # Tamanho das camadas ocultas
                learning_rate_init=learning_rate_init,  # Taxa de aprendizado inicial
                max_iter=4000,                          # Número máximo de iterações para o treinamento
                random_state=42                         # Garantir consistência nos resultados
            )
        )

        # Treina o modelo com os dados de treino
        model.fit(X_train, y_train)

        # Faz previsões com os dados de validação
        y_pred = model.predict(X_val)

        # Calcula as métricas de desempenho e guarda os resultados
        accuracy_scores.append(accuracy_score(y_val, y_pred))
        precision_scores.append(precision_score(y_val, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_val, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_val, y_pred, zero_division=0))

    # Mostra os resultados da média e desvio padrão das métricas
    print(f"\nhidden_layer_sizes={hidden_layer_sizes}, learning_rate_init={learning_rate_init}")
    print(f"Accuracy: Mean = {np.mean(accuracy_scores):.3f} Std = {np.std(accuracy_scores):.3f}")
    print(f"Precision: Mean = {np.mean(precision_scores):.3f} Std = {np.std(precision_scores):.3f}")
    print(f"Recall: Mean = {np.mean(recall_scores):.3f} Std = {np.std(recall_scores):.3f}")
    print(f"F1 Score: Mean = {np.mean(f1_scores):.3f} Std = {np.std(f1_scores):.3f}")

# Teste manual 1
print()
print("Teste manual 1:")
run_nn_cv(hidden_layer_sizes=(50,), learning_rate_init=0.001)

# Teste manual 2
print()
print("Teste manual 2:")
run_nn_cv(hidden_layer_sizes=(100,), learning_rate_init=0.001)

# Teste manual 3
print()
print("Teste manual 3:")
run_nn_cv(hidden_layer_sizes=(50, 50), learning_rate_init=0.001)


Teste manual 1:

hidden_layer_sizes=(50,), learning_rate_init=0.001
Accuracy: Mean = 0.772 Std = 0.005
Precision: Mean = 0.870 Std = 0.038
Recall: Mean = 0.474 Std = 0.026
F1 Score: Mean = 0.613 Std = 0.013

Teste manual 2:

hidden_layer_sizes=(100,), learning_rate_init=0.001
Accuracy: Mean = 0.776 Std = 0.004
Precision: Mean = 0.901 Std = 0.033
Recall: Mean = 0.465 Std = 0.030
F1 Score: Mean = 0.612 Std = 0.019

Teste manual 3:

hidden_layer_sizes=(50, 50), learning_rate_init=0.001
Accuracy: Mean = 0.784 Std = 0.006
Precision: Mean = 0.859 Std = 0.047
Recall: Mean = 0.525 Std = 0.047
F1 Score: Mean = 0.649 Std = 0.025


> * Observou-se que a arquitetura com duas camadas ocultas de 50 neurónios cada (hidden_layer_sizes=(50, 50)) e taxa de aprendizagem de 0.001 apresentou melhores resultados nas métricas de desempenho, especialmente na F1 Score.
>  
> * Isto reflete um bom equilíbrio entre precisão e sensibilidade, sugerindo que uma arquitetura um pouco mais complexa contribui positivamente para a capacidade preditiva do modelo neste problema.

In [ ]:
nn_model = make_pipeline(StandardScaler(), MLPClassifier(max_iter=4000))

# Hiperparâmetros do modelo de rede neural
nn_model_params = {
    'mlpclassifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'mlpclassifier__learning_rate_init': [0.001, 0.01]
}

# Otimizar hiperparâmetros do MLPClassifier
grid = GridSearchCV(nn_model, nn_model_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Ajustar (treinar) o modelo com os dados
grid.fit(X, y)

# Obter os melhores parâmetros e a melhor pontuação
nn_best_params = grid.best_params_
nn_best_score = -grid.best_score_.round(2)
print(f"Best Neural Network Parameters: {nn_best_params}")
print(f"Best Score: {dt_best_score}")

# Treinar o modelo com os melhores parâmetros
nn_model.set_params(**nn_best_params)

# Avaliar desempenho do modelo 
scores = cross_validate(nn_model, X, y, cv=5, scoring=scoring, n_jobs=-1)

# Calcular as métricas de avaliação
nn_metrics = {}
for metric in scoring.keys():
    nn_metrics[metric] = (np.mean(scores[f'test_{metric}']), np.std(scores[f'test_{metric}']))

# Exibir as métricas de avaliação do modelo
print("Neural Network Evaluation:")
for metric, (mean, std) in nn_metrics.items():
    print(f"{metric}: Mean = {mean:.3f}, Std = {std:.3f}")

neural_network = nn_model

Best Neural Network Parameters: {'mlpclassifier__hidden_layer_sizes': (50, 50), 'mlpclassifier__learning_rate_init': 0.01}
Best Score: 0.22
Neural Network Evaluation:
Accuracy: Mean = 0.757, Std = 0.018
Sensitivity: Mean = 0.456, Std = 0.098
Specificity: Mean = 0.942, Std = 0.038
F1: Mean = 0.581, Std = 0.068


> * Melhor configuração a rede com camadas ocultas de tamanho (50, 50) e uma taxa de aprendizagem inicial de 0.01;
>
> * MAE de 0.23, valor idêntico ao obtido pela árvore de decisão otimizada anteriormente;
>
> * Este resultado sugere que, em termos médios, o modelo neural apresenta um desempenho semelhante ao modelo mais simples, apesar da sua maior complexidade.

#### c. Modelo SVM

In [ ]:
# Divide os dados em 5 conjuntos (folds), com os dados baralhados antes
# O parâmetro random_state assegura que a divisão é reprodutível (mesma em cada execução)
# Usado para criar os folds de validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Função para executar a validação cruzada com os hiperparâmetros (kernel_type) definidos
def run_svm_cv(kernel_type):
    # Listas para armazenar os resultados das métricas em cada fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Percorre os 5 folds da validação cruzada
    for train_idx, val_idx in kf.split(X):
        # Divide os dados em conjuntos de treino e validação
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Cria e treina o modelo SVM com o kernel especificado
        model = make_pipeline(
            StandardScaler(),
            SVC(kernel=kernel_type, random_state=42)
        )

        # Treina o modelo com os dados de treino
        model.fit(X_train, y_train)

        # Faz previsões com os dados de validação
        y_pred = model.predict(X_val)

        # Calcula as métricas de desempenho e guarda os resultados
        accuracy_scores.append(accuracy_score(y_val, y_pred))
        precision_scores.append(precision_score(y_val, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_val, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_val, y_pred, zero_division=0))

    # Mostra os resultados da média e desvio padrão das métricas
    print(f"\nKernel: {kernel_type}")
    print(f"Accuracy: Mean = {np.mean(accuracy_scores):.3f} Std = {np.std(accuracy_scores):.3f}")
    print(f"Precision: Mean = {np.mean(precision_scores):.3f} Std = {np.std(precision_scores):.3f}")
    print(f"Recall: Mean = {np.mean(recall_scores):.3f} Std = {np.std(recall_scores):.3f}")
    print(f"F1 Score: Mean = {np.mean(f1_scores):.3f} Std = {np.std(f1_scores):.3f}")

# Teste manual 1: kernel linear
print("Teste manual 1:")
run_svm_cv(kernel_type='linear')

# Teste manual 2: kernel RBF
print()
print("Teste manual 2:")
run_svm_cv(kernel_type='rbf')

Teste manual 1:

Kernel: linear
Accuracy: Mean = 0.691 Std = 0.004
Precision: Mean = 0.996 Std = 0.002
Recall: Mean = 0.189 Std = 0.005
F1 Score: Mean = 0.318 Std = 0.007

Teste manual 2:

Kernel: rbf
Accuracy: Mean = 0.691 Std = 0.004
Precision: Mean = 1.000 Std = 0.001
Recall: Mean = 0.189 Std = 0.005
F1 Score: Mean = 0.318 Std = 0.007


> * O modelo SVM com kernel RBF apresentou melhor desempenho, com F1 Score superior ao do kernel linear
> 
> * Maior capacidade de modelar padrões não lineares presentes nos dados.

In [ ]:
svm_model = make_pipeline(StandardScaler(), SVC())

# Hiperparâmetros do modelo SVM
svm_model_params = {
    'svc__kernel': ['linear', 'rbf'],
}

# Otimizar hiperparâmetros do SVM
grid = GridSearchCV(svm_model, svm_model_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Ajustar (treinar) o modelo com os dados
grid.fit(X, y)

# Obter os melhores parâmetros e a melhor pontuação
svm_best_params = grid.best_params_
svm_best_score = -grid.best_score_.round(2)
print(f"Best SVM Parameters: {svm_best_params}")
print(f"Best Score: {dt_best_score}")

# Treinar o modelo com os melhores parâmetros
svm_model.set_params(**svm_best_params)

# Avaliar desempenho do modelo
scores = cross_validate(svm_model, X, y, cv=5, scoring=scoring, n_jobs=-1)

# Calcular as métricas de avaliação
svm_metrics = {}
for metric in scoring.keys():
    svm_metrics[metric] = (np.mean(scores[f'test_{metric}']), np.std(scores[f'test_{metric}']))

# Exibir as métricas de avaliação do modelo
print("SVM Evaluation:")
for metric, (mean, std) in svm_metrics.items():
    print(f"{metric}: Mean = {mean:.3f}, Std = {std:.3f}")

svm = svm_model

Best SVM Parameters: {'svc__kernel': 'rbf'}
Best Score: 0.22
SVM Evaluation:
Accuracy: Mean = 0.691, Std = 0.015
Sensitivity: Mean = 0.189, Std = 0.040
Specificity: Mean = 1.000, Std = 0.000
F1: Mean = 0.316, Std = 0.059


> * RBF apresenta melhor desempenho 
>
> * Atingiu um MAE de 0.31 e este resultado indica um desempenho inferior comparado com os modelos anteriores;
> 
> * Este resultado significa que o SVM não foi tão eficaz na previsão da variável de interesse.

#### d. Modelo K-vizinhos-mais-próximos

In [ ]:
# Divide os dados em 5 conjuntos (folds), com os dados baralhados antes.
# O parâmetro random_state assegura que a divisão é reprodutível (mesma em cada execução).
# Usado para criar os folds de validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Função para executar a validação cruzada com os hiperparâmetros (n_neighbors) definidos
def run_knn_cv(n_neighbors):
    # Listas para armazenar os resultados das métricas em cada fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Percorre os 5 folds da validação cruzada
    for train_idx, val_idx in kf.split(X):
        # Divide os dados em conjuntos de treino e validação
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Cria e treina o modelo KNN com o número de vizinhos especificado
        model = make_pipeline(
            StandardScaler(),
            KNeighborsClassifier(n_neighbors=n_neighbors)
        )

        # Treina o modelo com os dados de treino
        model.fit(X_train, y_train)

        # Faz previsões com os dados de validação
        y_pred = model.predict(X_val)

        # Faz previsões com os dados de validação
        accuracy_scores.append(accuracy_score(y_val, y_pred))
        precision_scores.append(precision_score(y_val, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_val, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_val, y_pred, zero_division=0))

    # Mostra os resultados da média e desvio padrão das métricas
    print(f"\nn_neighbors: {n_neighbors}")
    print(f"Accuracy: Mean = {np.mean(accuracy_scores):.3f} Std = {np.std(accuracy_scores):.3f}")
    print(f"Precision: Mean = {np.mean(precision_scores):.3f} Std = {np.std(precision_scores):.3f}")
    print(f"Recall: Mean = {np.mean(recall_scores):.3f} Std = {np.std(recall_scores):.3f}")
    print(f"F1 Score: Mean = {np.mean(f1_scores):.3f} Std = {np.std(f1_scores):.3f}")

# Teste manual 1: k = 3
print("Teste manual 1:")
run_knn_cv(n_neighbors=3)

# Teste manual 2: k = 5
print()
print("Teste manual 2:")
run_knn_cv(n_neighbors=5)

# Teste manual 3: k = 10
print()
print("Teste manual 3:")
run_knn_cv(n_neighbors=10)

Teste manual 1:

n_neighbors: 3
Accuracy: Mean = 0.797 Std = 0.005
Precision: Mean = 0.803 Std = 0.010
Recall: Mean = 0.621 Std = 0.006
F1 Score: Mean = 0.700 Std = 0.006

Teste manual 2:

n_neighbors: 5
Accuracy: Mean = 0.767 Std = 0.004
Precision: Mean = 0.793 Std = 0.011
Recall: Mean = 0.525 Std = 0.008
F1 Score: Mean = 0.632 Std = 0.007

Teste manual 3:

n_neighbors: 10
Accuracy: Mean = 0.756 Std = 0.004
Precision: Mean = 0.848 Std = 0.014
Recall: Mean = 0.439 Std = 0.010
F1 Score: Mean = 0.578 Std = 0.008


> * O valor k=10 proporcionou o melhor equilíbrio entre precisão e estabilidade das métricas; 
> 
> * Isso indica que, para este conjunto de dados, um número maior de vizinhos permite uma melhor generalização do modelo.

In [ ]:
knn_model = make_pipeline(StandardScaler(), KNeighborsClassifier())

# Hiperparâmetros do modelo KNN
knn_model_params = {
    'kneighborsclassifier__n_neighbors': [3, 5, 10],
}

# Otimizar hiperparâmetros do KNeighborsClassifier
grid = GridSearchCV(knn_model, knn_model_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Ajustar (treinar) o modelo com os dados
grid.fit(X, y)

# Obter os melhores parâmetros e a melhor pontuação
knn_best_params = grid.best_params_
knn_best_score = -grid.best_score_.round(2)
print(f"Best KNN Parameters: {knn_best_params}")
print(f"Best Score: {dt_best_score}")

# Treinar o modelo com os melhores parâmetros
knn_model.set_params(**knn_best_params)

# Avaliar desempenho do modelo
scores = cross_validate(knn_model, X, y, cv=5, scoring=scoring, n_jobs=-1)

# Calcular as métricas de avaliação
knn_metrics = {}
for metric in scoring.keys():
    knn_metrics[metric] = (np.mean(scores[f'test_{metric}']), np.std(scores[f'test_{metric}']))

# Exibir as métricas de avaliação do modelo
print("KNN Evaluation:")
for metric, (mean, std) in knn_metrics.items():
    print(f"{metric}: Mean = {mean:.3f}, Std = {std:.3f}")

knn = knn_model

Best KNN Parameters: {'kneighborsclassifier__n_neighbors': 10}
Best Score: 0.22
KNN Evaluation:
Accuracy: Mean = 0.728, Std = 0.011
Sensitivity: Mean = 0.429, Std = 0.047
Specificity: Mean = 0.912, Std = 0.037
F1: Mean = 0.544, Std = 0.028


> * O melhor número de vizinhos (n_neighbors) foi o 10;
>
> * O MAE foi de 0.27, o que significa que, apesar de ter demonstrado uma capacidade preditiva razoável, a sua simplicidade e, por ser, um método baseado em instâncias, podem comprometer a sua capacidade de generalização.

# 4.3.4 Modelo com melhor desempenho entre os dois melhores

> O teste t de Student foi escolhido, pois compara diretamente duas séries de resultados dependentes (folds iguais) e com este é possível observar as diferenças entre os modelos.

In [ ]:
# Definir métricas de avaliação a usar na cross_validate
scoring = {
    'Accuracy': 'accuracy',
    'Sensitivity': 'recall',
    'Specificity': specificity_scorer,
    'F1': 'f1'
}

# Modelos a comparar
# model_a: árvore de decisão (decision_tree)
# model_b: rede neural (neural_network)
model_a = decision_tree
model_b = neural_network

# Prepara as variáveis preditoras (X) e a variável alvo (y)
# A variável RespDisease é o alvo
X = df_num.drop(columns=['RespDisease'])
y = df_num['RespDisease']

# Avaliar desempenho dos modelos
scores_a = cross_validate(model_a, X, y, cv=5, scoring=scoring, n_jobs=-1)
scores_b = cross_validate(model_b, X, y, cv=5, scoring=scoring, n_jobs=-1)

# Compara os desempenhos das duas abordagens para cada métrica de avaliação
# Utiliza o teste t de Student para amostras emparelhadas (ttest_rel), pois os folds são iguais
# Avalia se as diferenças observadas são estatisticamente significativas (nível de significância de 5%)
results = []
for metric in scoring.keys():
    # Valores do modelo A (árvore)
    a_scores = scores_a[f'test_{metric}']

    # Valores do modelo B (rede neural)
    b_scores = scores_b[f'test_{metric}']

    # teste t Student
    t_stat, p_val = ttest_rel(a_scores, b_scores)

    # Diferença significativa se p < 0.05
    significant = p_val < 0.05

    results.append({
        'Metric': metric,
        'Model A Mean': np.mean(a_scores),
        'Model B Mean': np.mean(b_scores),
        't-statistic': t_stat,
        'p-value': p_val,
        'Significant (α=0.05)': significant
    })

# Mostrar resultados 
pd.DataFrame(results)

,Metric,Model A Mean,Model B Mean,t-statistic,p-value,Significant (α=0.05)
0,Accuracy,0.781644,0.766585,1.956553,0.122036,False
1,Sensitivity,0.510966,0.552645,-2.971801,0.041071,True
2,Specificity,0.948488,0.898458,2.611616,0.059319,False
3,F1,0.640129,0.643073,-0.445202,0.679205,False


> * Accuracy: não existiu grande diferença significativa, entre os dois modelos (p=0.122).  
> 
> * Sensitivity: a Rede Neuronal apresentou um melhor desempenho, com uma diferença estatisticamente significativa (p=0.041), isto significa que este é melhor a identificar corretamente pacientes com doenças respiratórias.
> 
> * Specificity: foi maior na Árvore de Decisão, significando que este pode ser mais eficaz a reconhecer negativos verdadeiros.
> 
> * F1: não revelou diferenças relevantes entre os modelos (p=0.697), indicando um desempenho igual entre falsos positivos e falsos negativos.


# 4.3.5 Comparação dos resultados do modelo

> * Accuracy: a Árvore de Decisão teve melhor desempenho, enquanto que o SVM teve pior, ou seja, este último classificou incorretamente a maior parte das instâncias.
> 
> * Sensitivity: a Árvore de decisão teve melhor desempenho, mas o SVM teve uma sensibilidade baixa, o que indica que ocorrem falhas na deteção de casos positivos.
> 
> * Specificity: o SVM teve melhor desempenho, mas isto ocorreu às custas de uma sensibilidade muito baixa, ou seja, não detetou positivos; o pior foi a Rede Neuronal, apesar de ainda ser bastante alta.
>
> * F1: a Árvore de Decisão teve melhor desempenho, ao contrário do SVM que obteve um F1 muito baixo, o que significa um desempenho desequilibrado.
>
> Conclusão: o melhor modelo é a Árvore de Decisão, uma vez que teve melhor desempenho em Accuracy, Sensitivity e F1 e o pior modelo é o SVM, pois falhou na identificação de positivos, ou seja, risco de ignorar pacientes doentes
